In [ ]:
using CUDA

# Check if CUDA is available
if CUDA.has_cuda()
    println("CUDA-enabled GPU is available.")
else
    println("CUDA-enabled GPU is not available.")
end

In [ ]:
# Check if CUDA is available
if CUDA.has_cuda()
    println("CUDA-enabled GPU is available.")
    # Optionally, you can get more information about the GPU
    device = CUDA.device()
    println("GPU Device: ", device)
    println("GPU Name: ", CUDA.name(device))
else
    println("CUDA-enabled GPU is not available.")
end

In [1]:
using PyCall

py"""
import sys
sys.path.append(".")
from entity_module import Entity
import torch
import redis
"""

# Import the Python module
entity_module = pyimport("entity_module")

# Connect to Redis
r = entity_module.redis.Redis(host="localhost", port=6379, db=0)

# Create tensors
tensor1 = entity_module.torch.randint(0, 2, (1024, 64), dtype=entity_module.torch.int64).cuda()
tensor2 = entity_module.torch.randint(0, 2, (1024, 64), dtype=entity_module.torch.int64).cuda()

# Create entities with identifiers
entity1 = entity_module.Entity(tensor1, identifier="entity1")
entity2 = entity_module.Entity(tensor2, identifier="entity2")

# Save entities to Redis
entity1.save_to_redis(r)
entity2.save_to_redis(r)

# Perform union and create a new entity
union_result = entity_module.elementwise_union(entity1.hllset, entity2.hllset)
entity_union = entity_module.Entity(union_result, identifier="entity_union", references=[entity1, entity2])
entity_union.save_to_redis(r)
println("Union Result:\n", union_result)

# Perform intersection and create a new entity
intersection_result = entity_module.elementwise_intersection(entity1.hllset, entity2.hllset)
entity_intersection = entity_module.Entity(intersection_result, identifier="entity_intersection", references=[entity1, entity2])
entity_intersection.save_to_redis(r)
println("Intersection Result:\n", intersection_result)

# Retrieve entities from Redis
retrieved_entity_union = entity_module.Entity.load_from_redis(r, "entity_union")
println("Retrieved Entity Union:\n", retrieved_entity_union.hllset)

retrieved_entity_intersection = entity_module.Entity.load_from_redis(r, "entity_intersection")
println("Retrieved Entity Intersection:\n", retrieved_entity_intersection.hllset)

Union Result:
PyObject tensor([[0, 1, 1,  ..., 1, 1, 1],
        [1, 0, 1,  ..., 0, 0, 1],
        [0, 1, 1,  ..., 0, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 0, 0],
        [0, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 0, 1, 1]], device='cuda:0')
Intersection Result:
PyObject tensor([[0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0],
        [1, 0, 0,  ..., 0, 0, 1]], device='cuda:0')
Retrieved Entity Union:
PyObject tensor([[0, 1, 1,  ..., 1, 1, 1],
        [1, 0, 1,  ..., 0, 0, 1],
        [0, 1, 1,  ..., 0, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 0, 0],
        [0, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 0, 1, 1]], device='cuda:0')
Retrieved Entity Intersection:
PyObject tensor([[0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 1, 0,  .

/home/alexmy/JULIA/SGS/SGS/./entity_module.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(buffer)
